Informações completas sobre o pedido


In [1]:
import psycopg 

Conexão com o banco 

In [18]:
from psycopg import OperationalError
try:
    northwind = psycopg.connect(
        host='localhost',
        dbname='postgres', 
        user = 'postgres', 
        password = 'root'
    )
    print ("Conexão executada com sucesso")
except OperationalError as e:
    print(e)

Conexão executada com sucesso


Inserção do pedido


In [30]:
# TABELA orders

# nome do cliente | custumerid
custumerid = "LILAS"
# nome do vendednor | employeeid
employeeid = 9
# dados do pedido |
orderdate = "1994-11-16 00:00:00"
requireddate = "1994-11-28 00:00:00"
shippeddate ="1994-11-28 00:00:00"
freight =12.7500
shipname ="LILA-Supermercado"
shipaddress ="Carrera 52 con Ave. Bolar #65-98 Llano Largo"
shipcity ="Lyon"
shipregion ="Co. Cork"
shippostalcode ="69004"
shipcountry ="France"
shipperid = 1

# TABELA order_details
# itens do pedido |
productid = 42
unitprice = 9.8000
quantity = 10
discount = 0.0000

try:
    sessao = northwind.cursor()

    sessao.execute("SELECT MAX(orderid) FROM northwind.orders")
    last_id_order = sessao.fetchone()[0]
    new_order_id = int(last_id_order) + 1
    print(new_order_id)

    insert_order = f"""
        INSERT INTO northwind.orders
        (orderid, customerid, employeeid, orderdate, requireddate, shippeddate, freight, shipname, shipaddress, shipcity, shipregion, shippostalcode,   shipcountry, shipperid)
        VALUES ({new_order_id}, {new_order_id}, '{employeeid}', '{orderdate}', '{requireddate}', '{shippeddate}', {freight}, '{shipname}', '{shipaddress}', '{shipcity}', '{shipregion}', '{shippostalcode}', '{shipcountry}', {shipperid});
    """
            
    insert_order_details = f"""
        INSERT INTO northwind.order_details(orderid, productid, unitprice, quantity, discount)
        VALUES ({new_order_id}, {productid}, {unitprice}, {quantity}, {discount});
    """

    sessao.execute(insert_order)
    sessao.execute(insert_order_details)

    northwind.commit()
except Exception as e:
    print("Erro durante a execução:", e)
    northwind.rollback()





11078


In [17]:
northwind.close()

Informações comletas de um pedido 

In [58]:
order = 11077

try:
    sessao = northwind.cursor()


    select_orders = f"""
        SELECT 
            ord.orderdate,
            ord.orderid,
            cus.companyname,
            emp.firstname,
            emp.lastname	
        FROM northwind.orders ord
        LEFT JOIN northwind.customers cus ON ord.customerid = cus.customerid
        LEFT JOIN northwind.employees emp ON ord.employeeid = emp.employeeid 
        WHERE ord.orderid = {order}
    """

    select_orders_details = f"""
        SELECT 
            pro.productname,
            ode.quantity,
            ode.unitprice
        FROM northwind.order_details ode
        LEFT JOIN northwind.products pro ON ode.productid = pro.productid 
        WHERE ode.orderid = {order}
    """

    sessao.execute(select_orders)
    print("PEDIDO")
    metadata_colunas_orders = [desc[0] for desc in sessao.description]
    for prod in sessao:
        dicionario = dict(zip(metadata_colunas_orders, prod))

        print(f"""
        Numero pedido: {dicionario['orderid']}
        Data Pedido: {dicionario['orderdate']}
        Nome do Cliente: {dicionario['companyname']}
        Nome do vendedor: {dicionario['firstname']} {dicionario['lastname']}
        """)
    

    sessao.execute(select_orders_details)
    print("\nITENS DO PEDIDO")
    metadata_colunas_orders_details = [desc[0] for desc in sessao.description]
    for i, prod in enumerate(sessao):
        dicionario = dict(zip(metadata_colunas_orders_details, prod))
        print(f"""\t{i+1}. Produto: {dicionario['productname']} - Quantidade: {dicionario['quantity']} - Preço: {dicionario['unitprice']}""")
    
    northwind.commit()
except Exception as e:
    print("Erro durante a execução:", e)
    northwind.rollback()

PEDIDO

        Numero pedido: 11077
        Data Pedido: 1996-06-05 00:00:00
        Nome do Cliente: Rattlesnake Canyon Grocery
        Nome do vendedor: Nancy Davolio
        

ITENS DO PEDIDO
	1. Produto: Chang - Quantidade: 24 - Preço: 19.0000
	2. Produto: Aniseed Syrup - Quantidade: 4 - Preço: 10.0000
	3. Produto: Chef Anton's Cajun Seasoning - Quantidade: 1 - Preço: 22.0000
	4. Produto: Grandma's Boysenberry Spread - Quantidade: 1 - Preço: 25.0000
	5. Produto: Uncle Bob's Organic Dried Pears - Quantidade: 1 - Preço: 30.0000
	6. Produto: Northwoods Cranberry Sauce - Quantidade: 2 - Preço: 40.0000
	7. Produto: Ikura - Quantidade: 1 - Preço: 31.0000
	8. Produto: Queso Manchego La Pastora - Quantidade: 2 - Preço: 38.0000
	9. Produto: Konbu - Quantidade: 4 - Preço: 6.0000
	10. Produto: Tofu - Quantidade: 1 - Preço: 23.2500
	11. Produto: Pavlova - Quantidade: 2 - Preço: 17.4500
	12. Produto: Sir Rodney's Marmalade - Quantidade: 1 - Preço: 81.0000
	13. Produto: TunnbrÃ¶d - Quantidade: 

Ranking dos funcionários por intervalo de tempo

In [60]:
data_inicio = '1996-05-30 00:00:00' 
data_fim = '1996-06-05 00:00:00'

try:
    sessao = northwind.cursor()
    select_ranking = f"""
        SELECT 
            emp.firstname,
            emp.lastname,
            SUM(ode.quantity) AS soma_qtd,
            SUM(ode.unitprice * ode.quantity) AS valor_total
        FROM northwind.orders ord
        LEFT JOIN northwind.order_details ode ON ord.orderid = ode.orderid
        LEFT JOIN northwind.employees emp ON ord.employeeid = emp.employeeid
        WHERE ord.orderdate BETWEEN '1996-05-30 00:00:00' AND '1996-06-05 00:00:00'
        GROUP BY emp.firstname, emp.lastname
        ORDER BY valor_total DESC;
    """

    sessao.execute(select_ranking)

    print("\nRANKING FUNCIONARIOS")
    metadata_colunas_orders_details = [desc[0] for desc in sessao.description]
    for i, prod in enumerate(sessao):
        dicionario = dict(zip(metadata_colunas_orders_details, prod))
        print(f"""\t{i+1}. Nome: {dicionario['firstname']} {dicionario['lastname']} - Quantidade: {dicionario['soma_qtd']} - Valor total: {dicionario['valor_total']}""")
    
    northwind.commit()
except Exception as e:
    print("Erro durante a execução:", e)
    northwind.rollback()




RANKING FUNCIONARIOS
	1. Nome: Margaret Peacock - Quantidade: 287 - Valor total: 7293.0000
	2. Nome: Nancy Davolio - Quantidade: 299 - Valor total: 7053.7500
	3. Nome: Laura Callahan - Quantidade: 138 - Valor total: 3223.3600
	4. Nome: Andrew Fuller - Quantidade: 154 - Valor total: 2439.5000
	5. Nome: Janet Leverling - Quantidade: 100 - Valor total: 1445.5000
	6. Nome: Robert King - Quantidade: 94 - Valor total: 1173.0500
